In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

## 1. Load the Dataset

In [2]:
# Load the dataset
file_path = "/Users/mason/Desktop/CART_manufacturing_prediction/Dataset01.csv"  
df = pd.read_csv(file_path)

### 1.1 Missing Values in the Data set

In [3]:
pd.set_option('display.max_rows', None)
#  Count non-null values for each column
non_null_counts = df.notnull().sum()

# Sort by ascending order
sorted_non_null_counts = non_null_counts.sort_values(ascending=True)

# Display the top 20 columns with the least non-null counts and their types
least_non_null_columns = sorted_non_null_counts
column_types = df[least_non_null_columns.index].dtypes

# Combine non-null counts and types for display
result = pd.DataFrame({
    "Non-Null Count": least_non_null_columns,
    "Type": column_types
})

result

,Non-Null Count,Type
Step03Var12,12,object
Step03Var11,69,object
Step03Var10,69,object
Step00Var5,107,float64
Step03Var5,527,object
Step04Var10,1096,object
Step04Var4,1392,object
Step04Var3,1392,object
Step04Var9,1439,object
Step04Var8,1441,object


In [4]:
# Drop rows where TargetVar is missing
df = df.dropna(subset=['TargetVar'])

# Confirm changes
print(f"Remaining rows: {df.shape}")

Remaining rows: (4529, 64)


In [5]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extract new date features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday

print(df.head())

     MaskID     TypeID       Date  Step00Var1  Step00Var2  Step00Var3  \
0  ID000000  PATIENT_A 2022-03-12        12.0  32400000.0        97.6   
1  ID000001  PATIENT_A 2022-03-11        11.0  52200000.0        93.7   
2  ID000002  PATIENT_A 2022-03-13        12.0  44700000.0        94.2   
3  ID000003  PATIENT_A 2022-03-13        16.0  58600000.0        96.9   
4  ID000004  PATIENT_A 2022-03-14        14.0  74100000.0        99.1   

   Step00Var4  Step00Var5  Step00Var6  Step00Var7  ...  Step04Var7  \
0        66.8         NaN         4.0         6.0  ...         NaN   
1        65.4         NaN         4.0         6.1  ...         NaN   
2        65.5         NaN         4.0         6.1  ...         NaN   
3        61.7         NaN         3.8         6.2  ...         NaN   
4        64.4         NaN         4.0         6.2  ...          4+   

         Step04Var8 Step04Var9 Step04Var10 Step04Var12 TargetVar  Year Month  \
0               NaN        NaN         NaN        91.7      

### 1.2 Insignificant Columns

In [6]:
# Columns like Step03Var12 (12 non-null), Step03Var11 (69 non-null), etc., provide very little information relative to the dataset's size. 
# Drop columns with very few non-null values
low_non_null_cols = ['Step03Var12', 'Step03Var11', 'Step03Var10', 'Step00Var5', 'Step03Var5']
df = df.drop(columns=low_non_null_cols)

# Confirm changes
print(f"Remaining columns: {df.shape}")

Remaining columns: (4529, 63)


### 1.3 Impute the Missing Values
#### KNN 

In [7]:
# Select object columns
object_cols = df.select_dtypes(include=['object']).columns

# Display unique values in each object column
for col in object_cols:
    print(f"{col}: {df[col].unique()}")

MaskID: ['ID000000' 'ID000001' 'ID000002' ... 'ID004583' 'ID004584' 'ID004585']
TypeID: ['PATIENT_A' 'PATIENT_B' 'PATIENT_C' 'PATIENT_D' 'PATIENT_E']
Step00Var9: ['51' '38' '17' '27' '16' '71' '52' '42' '18' '64' '79' '73' '30' '59'
 '39' '12' '33' '31' '45' '44' '34' '15' '58' '48' '28' '68' '32' '63' '9'
 '20' '13' '36' '26' '91' '60' '35' '47' '57' '55' '14' '49' '56' '22'
 '10' '19' '54' '7' '70' '65' '92' '24' '43' '40' '72' '25' '21' '41' '62'
 '37' '29' '50' '53' '5' '23' '66' '67' '46' '86' '11' nan '61' '69' '78'
 '75' '6' '74' '80' '8' '85' '81' '76' '77' '84' 'Not Reported' '82' '83'
 '0' '87' '89' '4' '3' '90']
Step00Var10: ['45' '59' '81' '71' '83' '24' '47' '58' '77' '31' '17' '66' '37' '73'
 '63' '51' '53' '60' '82' '38' '49' '30' '27' '64' '33' '67' '78' '55'
 '29' '22' '40' '7' '35' '62' '43' '50' '42' '85' '34' '48' '74' '36' '76'
 '87' '69' '26' '92' '46' '32' '41' '75' '44' '57' '61' '79' '72' '54'
 '80' '70' '28' '65' '52' '91' '39' '68' '12' '84' nan '25' '19' '56

In [8]:
from sklearn.impute import KNNImputer

# Exclude non-relevant columns
excluded_cols = ['Date', 'Year', 'Month', 'Day', 'Weekday', 'MaskID', 'TypeID']
numerical_cols = [col for col in df.select_dtypes(include=[float]).columns if col not in excluded_cols]

# Select only numerical columns for imputation
numerical_data = df[numerical_cols]

# Apply KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)
imputed_numerical_data = knn_imputer.fit_transform(numerical_data)

# Update the DataFrame with imputed data
df[numerical_cols] = imputed_numerical_data

df.shape

(4529, 63)

In [9]:
pd.set_option('display.max_rows', None)

# Identify remaining object columns
remaining_object_cols = df.select_dtypes(include=['object']).columns
print(f"Remaining object columns: {remaining_object_cols}")

# Display unique values for each remaining object column
for col in remaining_object_cols:
    print(f"Unique values in {col}: {df[col].unique()}")

df.columns

Remaining object columns: Index(['MaskID', 'TypeID', 'Step00Var9', 'Step00Var10', 'Step00Var11',
       'Step00Var12', 'Step00Var13', 'Step00Var14', 'Step00Var15',
       'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8', 'Step01Var9',
       'Step01Var10', 'Step01Var11', 'Step01Var12', 'Step03Var2', 'Step03Var3',
       'Step03Var4', 'Step03Var9', 'Step04Var2', 'Step04Var3', 'Step04Var4',
       'Step04Var6', 'Step04Var7', 'Step04Var8', 'Step04Var9', 'Step04Var10'],
      dtype='object')
Unique values in MaskID: ['ID000000' 'ID000001' 'ID000002' ... 'ID004583' 'ID004584' 'ID004585']
Unique values in TypeID: ['PATIENT_A' 'PATIENT_B' 'PATIENT_C' 'PATIENT_D' 'PATIENT_E']
Unique values in Step00Var9: ['51' '38' '17' '27' '16' '71' '52' '42' '18' '64' '79' '73' '30' '59'
 '39' '12' '33' '31' '45' '44' '34' '15' '58' '48' '28' '68' '32' '63' '9'
 '20' '13' '36' '26' '91' '60' '35' '47' '57' '55' '14' '49' '56' '22'
 '10' '19' '54' '7' '70' '65' '92' '24' '43' '40' '72' '25' '21' '41' '

Index(['MaskID', 'TypeID', 'Date', 'Step00Var1', 'Step00Var2', 'Step00Var3',
       'Step00Var4', 'Step00Var6', 'Step00Var7', 'Step00Var8', 'Step00Var9',
       'Step00Var10', 'Step00Var11', 'Step00Var12', 'Step00Var13',
       'Step00Var14', 'Step00Var15', 'Step01Var1', 'Step01Var2', 'Step01Var3',
       'Step01Var4', 'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8',
       'Step01Var9', 'Step01Var10', 'Step01Var11', 'Step01Var12',
       'Step01Var13', 'Step01Var14', 'Step01Var15', 'Step01Var16',
       'Step02Var1', 'Step02Var2', 'Step02Var3', 'Step02Var4', 'Step02Var5',
       'Step02Var6', 'Step03Var1', 'Step03Var2', 'Step03Var3', 'Step03Var4',
       'Step03Var6', 'Step03Var7', 'Step03Var8', 'Step03Var9', 'Step04Var1',
       'Step04Var2', 'Step04Var3', 'Step04Var4', 'Step04Var5', 'Step04Var6',
       'Step04Var7', 'Step04Var8', 'Step04Var9', 'Step04Var10', 'Step04Var12',
       'TargetVar', 'Year', 'Month', 'Day', 'Weekday'],
      dtype='object')

In [10]:
# Define columns to process (all object columns except MaskID and TypeID)
excluded_cols = ['MaskID', 'TypeID']  # Columns you don't want to encode
columns_to_process = [col for col in df.select_dtypes(include=['object']).columns if col not in excluded_cols]

numerical_like_cols = []
categorical_cols = []

# Dictionary to fix any known misspellings in categorical columns
category_mappings = {
    'Step04Var4': {
        'Greater than a DIme': 'Greater than a Dime',  # Correct typo
        'Less than a Dime': 'Less than a Dime',
        'Both (if multiple clumps)': 'Both (if multiple clumps)'
    }
}

# Apply mappings to fix categorical column values
for col, mapping in category_mappings.items():
    if col in df.columns:
        df[col] = df[col].replace(mapping)

# Process each column
for col in columns_to_process:
    unique_vals = df[col].dropna().unique()
    if len(unique_vals) > 20:  # Large number of unique values, likely numerical-like
        try:
            # Convert to numeric, replacing "Not Reported" or similar with NaN
            df[col] = pd.to_numeric(df[col].replace(['Not Reported'], np.nan), errors='coerce')
            numerical_like_cols.append(col)
        except ValueError:
            # If conversion fails, treat as categorical
            categorical_cols.append(col)
    else:
        categorical_cols.append(col)  # Treat as categorical if unique values are small

# Verify the classification of columns
print(f"Numerical-like columns: {numerical_like_cols}")
print(f"Categorical columns: {categorical_cols}")

# Handle one-hot encoding for categorical columns
if categorical_cols:
    print(f"One-hot encoding the following categorical columns: {categorical_cols}")
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Final summary
print("Numerical-like columns converted to numeric:", numerical_like_cols)
print("Categorical columns one-hot encoded:", categorical_cols)
print("Resulting DataFrame shape:", df.shape)
print("Resulting DataFrame columns:", df.columns)


Numerical-like columns: ['Step00Var9', 'Step00Var10', 'Step00Var11', 'Step00Var12', 'Step00Var13', 'Step00Var14', 'Step00Var15', 'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8', 'Step01Var9', 'Step01Var10', 'Step01Var12']
Categorical columns: ['Step01Var11', 'Step03Var2', 'Step03Var3', 'Step03Var4', 'Step03Var9', 'Step04Var2', 'Step04Var3', 'Step04Var4', 'Step04Var6', 'Step04Var7', 'Step04Var8', 'Step04Var9', 'Step04Var10']
One-hot encoding the following categorical columns: ['Step01Var11', 'Step03Var2', 'Step03Var3', 'Step03Var4', 'Step03Var9', 'Step04Var2', 'Step04Var3', 'Step04Var4', 'Step04Var6', 'Step04Var7', 'Step04Var8', 'Step04Var9', 'Step04Var10']
Numerical-like columns converted to numeric: ['Step00Var9', 'Step00Var10', 'Step00Var11', 'Step00Var12', 'Step00Var13', 'Step00Var14', 'Step00Var15', 'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8', 'Step01Var9', 'Step01Var10', 'Step01Var12']
Categorical columns one-hot encoded: ['Step01Var11', 'Step03Var2', 'Step03Var3

In [11]:
df.shape

(4529, 79)

In [12]:
pd.set_option('display.max_rows', None)
#  Count non-null values for each column
non_null_counts = df.notnull().sum()

# Sort by ascending order
sorted_non_null_counts = non_null_counts.sort_values(ascending=True)

# Display the top 20 columns with the least non-null counts and their types
least_non_null_columns = sorted_non_null_counts
column_types = df[least_non_null_columns.index].dtypes

# Combine non-null counts and types for display
result = pd.DataFrame({
    "Non-Null Count": least_non_null_columns,
    "Type": column_types
})

result

,Non-Null Count,Type
Step00Var13,4497,float64
Step00Var15,4497,float64
Step00Var14,4497,float64
Step00Var12,4497,float64
Step00Var11,4497,float64
Step00Var9,4497,float64
Step00Var10,4497,float64
Step01Var10,4499,float64
Step01Var9,4499,float64
Step01Var8,4499,float64


In [13]:
from sklearn.impute import KNNImputer
import pandas as pd

# Define the numerical columns with missing values
numerical_cols = [
    'Step00Var15', 'Step00Var14', 'Step00Var13', 'Step00Var12',
    'Step00Var11', 'Step00Var9', 'Step00Var10', 'Step01Var10',
    'Step01Var9', 'Step01Var8', 'Step01Var6', 'Step01Var5',
    'Step01Var12', 'Step01Var7'
]

# Initialize the KNN Imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 nearest neighbors for imputation

# Apply KNN Imputation on the specified columns
df[numerical_cols] = knn_imputer.fit_transform(df[numerical_cols])

# Check for remaining missing values in these columns
print("Missing values after KNN imputation:")
print(df[numerical_cols].isnull().sum())

Missing values after KNN imputation:
Step00Var15    0
Step00Var14    0
Step00Var13    0
Step00Var12    0
Step00Var11    0
Step00Var9     0
Step00Var10    0
Step01Var10    0
Step01Var9     0
Step01Var8     0
Step01Var6     0
Step01Var5     0
Step01Var12    0
Step01Var7     0
dtype: int64


In [14]:
pd.set_option('display.max_rows', None)
#  Count non-null values for each column
non_null_counts = df.notnull().sum()

# Sort by ascending order
sorted_non_null_counts = non_null_counts.sort_values(ascending=True)

# Display the top 20 columns with the least non-null counts and their types
least_non_null_columns = sorted_non_null_counts
column_types = df[least_non_null_columns.index].dtypes

# Combine non-null counts and types for display
result = pd.DataFrame({
    "Non-Null Count": least_non_null_columns,
    "Type": column_types
})

result

,Non-Null Count,Type
MaskID,4529,object
Step01Var11_Not Reported,4529,uint8
Step01Var11_7,4529,uint8
Step01Var11_6,4529,uint8
Step01Var11_4,4529,uint8
Step01Var11_3,4529,uint8
Step01Var11_2,4529,uint8
Step01Var11_1,4529,uint8
Step03Var2_Yes,4529,uint8
Weekday,4529,int64


In [15]:
df.head(10)

,MaskID,TypeID,Date,Step00Var1,Step00Var2,Step00Var3,Step00Var4,Step00Var6,Step00Var7,Step00Var8,...,Step04Var7_2-3,Step04Var7_4+,Step04Var7_Clumps dissipated,Step04Var8_Greater than a DIme,Step04Var8_Less than a Dime,Step04Var9_No Effect,Step04Var9_Some Effect,Step04Var10_2-3,Step04Var10_4+,Step04Var10_Clumps dissipated
0,ID000000,PATIENT_A,2022-03-12,12.0,32400000.0,97.6,66.8,4.0,6.0,2.236000e+09,...,0,0,0,0,0,0,0,0,0,0
1,ID000001,PATIENT_A,2022-03-11,11.0,52200000.0,93.7,65.4,4.0,6.1,3.497000e+09,...,0,0,0,0,0,0,0,0,0,0
2,ID000002,PATIENT_A,2022-03-13,12.0,44700000.0,94.2,65.5,4.0,6.1,3.040000e+09,...,0,0,0,0,0,0,0,0,0,0
3,ID000003,PATIENT_A,2022-03-13,16.0,58600000.0,96.9,61.7,3.8,6.2,3.750000e+09,...,0,0,0,0,0,0,0,0,0,0
4,ID000004,PATIENT_A,2022-03-14,14.0,74100000.0,99.1,64.4,4.0,6.2,4.891000e+09,...,0,1,0,0,1,1,0,1,0,0
5,ID000005,PATIENT_A,2022-03-15,10.0,44700000.0,92.7,62.1,3.8,6.1,2.861000e+09,...,0,0,0,0,0,0,0,0,0,0
6,ID000006,PATIENT_A,2022-03-15,12.0,35700000.0,94.1,64.7,4.0,6.2,2.392000e+09,...,0,0,0,0,0,0,0,0,0,0
7,ID000007,PATIENT_A,2022-03-16,10.0,35600000.0,92.3,65.3,4.0,6.1,2.385000e+09,...,0,0,0,0,0,0,0,0,0,0
8,ID000008,PATIENT_A,2022-03-17,11.0,35000000.0,97.6,66.3,4.0,6.0,2.380000e+09,...,0,0,0,0,0,0,0,0,0,0
9,ID000009,PATIENT_A,2022-03-17,13.0,32500000.0,88.6,66.6,4.0,6.0,2.243000e+09,...,0,1,0,0,1,0,1,0,0,0


In [16]:
from sklearn.preprocessing import MinMaxScaler

# Automatically select numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Exclude specific columns
excluded_cols = ['Year', 'Month', 'Day', 'Weekday']  # Columns to exclude
numerical_cols = [col for col in numerical_cols if col not in excluded_cols]

# Verify selected columns for normalization
print("Selected numerical columns for normalization (excluding specified columns):")
print(numerical_cols)

# Apply Min-Max Scaling
minmax_scaler = MinMaxScaler()
df[numerical_cols] = minmax_scaler.fit_transform(df[numerical_cols])

# Verify normalization
print("Summary statistics after normalization:")
print(df[numerical_cols].describe())

Selected numerical columns for normalization (excluding specified columns):
['Step00Var1', 'Step00Var2', 'Step00Var3', 'Step00Var4', 'Step00Var6', 'Step00Var7', 'Step00Var8', 'Step00Var9', 'Step00Var10', 'Step00Var11', 'Step00Var12', 'Step00Var13', 'Step00Var14', 'Step00Var15', 'Step01Var1', 'Step01Var2', 'Step01Var3', 'Step01Var4', 'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8', 'Step01Var9', 'Step01Var10', 'Step01Var12', 'Step01Var13', 'Step01Var14', 'Step01Var15', 'Step01Var16', 'Step02Var1', 'Step02Var2', 'Step02Var3', 'Step02Var4', 'Step02Var5', 'Step02Var6', 'Step03Var1', 'Step03Var6', 'Step03Var7', 'Step03Var8', 'Step04Var1', 'Step04Var5', 'Step04Var12', 'TargetVar']
Summary statistics after normalization:
        Step00Var1   Step00Var2   Step00Var3   Step00Var4   Step00Var6  \
count  4529.000000  4529.000000  4529.000000  4529.000000  4529.000000   
mean      0.258834     0.293751     0.831206     0.325148     0.302782   
std       0.143420     0.079884     0.097623   

In [17]:
df.head(10)

,MaskID,TypeID,Date,Step00Var1,Step00Var2,Step00Var3,Step00Var4,Step00Var6,Step00Var7,Step00Var8,...,Step04Var7_2-3,Step04Var7_4+,Step04Var7_Clumps dissipated,Step04Var8_Greater than a DIme,Step04Var8_Less than a Dime,Step04Var9_No Effect,Step04Var9_Some Effect,Step04Var10_2-3,Step04Var10_4+,Step04Var10_Clumps dissipated
0,ID000000,PATIENT_A,2022-03-12,0.363636,0.196746,0.938202,0.376471,0.3,0.166667,0.207003,...,0,0,0,0,0,0,0,0,0,0
1,ID000001,PATIENT_A,2022-03-11,0.272727,0.340548,0.828652,0.335294,0.3,0.333333,0.345849,...,0,0,0,0,0,0,0,0,0,0
2,ID000002,PATIENT_A,2022-03-13,0.363636,0.286077,0.842697,0.338235,0.3,0.333333,0.295530,...,0,0,0,0,0,0,0,0,0,0
3,ID000003,PATIENT_A,2022-03-13,0.727273,0.387029,0.918539,0.226471,0.2,0.500000,0.373706,...,0,0,0,0,0,0,0,0,0,0
4,ID000004,PATIENT_A,2022-03-14,0.545455,0.499601,0.980337,0.305882,0.3,0.500000,0.499339,...,0,1,0,0,1,1,0,1,0,0
5,ID000005,PATIENT_A,2022-03-15,0.181818,0.286077,0.800562,0.238235,0.2,0.333333,0.275820,...,0,0,0,0,0,0,0,0,0,0
6,ID000006,PATIENT_A,2022-03-15,0.363636,0.220713,0.839888,0.314706,0.3,0.500000,0.224180,...,0,0,0,0,0,0,0,0,0,0
7,ID000007,PATIENT_A,2022-03-16,0.181818,0.219987,0.789326,0.332353,0.3,0.333333,0.223409,...,0,0,0,0,0,0,0,0,0,0
8,ID000008,PATIENT_A,2022-03-17,0.272727,0.215629,0.938202,0.361765,0.3,0.166667,0.222858,...,0,0,0,0,0,0,0,0,0,0
9,ID000009,PATIENT_A,2022-03-17,0.454545,0.197473,0.685393,0.370588,0.3,0.166667,0.207774,...,0,1,0,0,1,0,1,0,0,0


In [18]:
print(df.columns)
print(df['Step04Var4_Greater than a Dime'].unique())


Index(['MaskID', 'TypeID', 'Date', 'Step00Var1', 'Step00Var2', 'Step00Var3',
       'Step00Var4', 'Step00Var6', 'Step00Var7', 'Step00Var8', 'Step00Var9',
       'Step00Var10', 'Step00Var11', 'Step00Var12', 'Step00Var13',
       'Step00Var14', 'Step00Var15', 'Step01Var1', 'Step01Var2', 'Step01Var3',
       'Step01Var4', 'Step01Var5', 'Step01Var6', 'Step01Var7', 'Step01Var8',
       'Step01Var9', 'Step01Var10', 'Step01Var12', 'Step01Var13',
       'Step01Var14', 'Step01Var15', 'Step01Var16', 'Step02Var1', 'Step02Var2',
       'Step02Var3', 'Step02Var4', 'Step02Var5', 'Step02Var6', 'Step03Var1',
       'Step03Var6', 'Step03Var7', 'Step03Var8', 'Step04Var1', 'Step04Var5',
       'Step04Var12', 'TargetVar', 'Year', 'Month', 'Day', 'Weekday',
       'Step01Var11_1', 'Step01Var11_2', 'Step01Var11_3', 'Step01Var11_4',
       'Step01Var11_6', 'Step01Var11_7', 'Step01Var11_Not Reported',
       'Step03Var2_Yes', 'Step03Var3_2-3', 'Step03Var3_4+',
       'Step03Var4_Greater than a DIme', 'Step03V

In [19]:
# Assuming the processed DataFrame is named 'df'
output_file_path = "/Users/mason/Desktop/CART_manufacturing_prediction/dataset1_cleaned.csv"

# Exporting the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

output_file_path

'/Users/mason/Desktop/CART_manufacturing_prediction/dataset1_cleaned.csv'